In [ ]:
import inspect
from mstrio.connection import Connection
import getpass
from mstrio.datasources import *
from mstrio.modeling.schema.table import *
from mstrio.modeling.expression import *
from mstrio.modeling.schema.fact import *
from mstrio.modeling import *

In [ ]:
PROJECT_NAME = "MicroStrategy Tutorial"
BASE_URL = "http://10.23.35.137:8080/MicroStrategyLibrary/api"
MSTR_USERNAME = "administrator"  # Insert your env username here
MSTR_PASSWORD = ""  # insert your mstr password here
PROJECT_ID = "B19DEDCC11D4E0EFC000EB9495D0F44F"  # Insert you project ID here
PUBLIC_FOLDER_ID = "98FE182C2A10427EACE0CD30B6768258"

In [ ]:
# Currently, supported authentication modes are Standard (the default), guest and LDAP.
# To use LDAP, add login_mode=16 when creating your Connection object:
conn = Connection(BASE_URL, MSTR_USERNAME, MSTR_PASSWORD, project_id=PROJECT_ID, login_mode=1)

# Discover Facts

## List facts

In [ ]:
list_facts(conn)

In [ ]:
list_facts(conn, name="Revenue")

In [ ]:
list_facts(conn, search_pattern=SearchPattern.EXACTLY, name="Revenue")

## Get Fact by Id or name

In [ ]:
fact = Fact(conn, id="8D67958411D3E4981000E787EC6DE8A4")

In [ ]:
# You cannot use initializing by name if there are more than one Fact object contains the name
fact = Fact(conn, name="Revenue")

## Properties of Fact object

In [ ]:
print(Fact.__doc__)

In [ ]:
fact.name

In [ ]:
fact.id

In [ ]:
fact.data_type

In [ ]:
fact.expressions

# Create Fact

To create a Fact, you use `Fact.create()` method.  

In [ ]:
print(Fact.create.__doc__)

## The Fact data

In [ ]:
# This is a simplified facts data, we will use in this example
fact_data =  {
      "information": {
        "name": "Revenue 2",
        "description": "Revenue 2",
        "destinationFolderId": "95C3B713318B43D490EE789BE27D298C"
      },
      "dataType": {
        "type": "decimal",
        "precision": 0,
        "scale": -2147483648
      },
      "expressions": [
        {
          "expression": {
            "tokens": [
              {
                "value": "TOT_DOLLAR_SALES"
              }
            ]
          },
          "tables": [
            {
              "subType": "logical_table",
              "name": "MNTH_CATEGORY_SLS"
            },
            {
              "subType": "logical_table",
              "name": "QTR_CATEGORY_SLS"
            },
            {
              "subType": "logical_table",
              "name": "CITY_CTR_SLS"
            },
            {
              "subType": "logical_table",
              "name": "CITY_MNTH_SLS"
            },
            {
              "subType": "logical_table",
              "name": "CITY_SUBCATEG_SLS"
            },
            {
              "subType": "logical_table",
              "name": "CUSTOMER_SLS"
            },
            {
              "subType": "logical_table",
              "name": "DAY_CTR_SLS"
            },
            {
              "subType": "logical_table",
              "name": "ITEM_EMP_SLS"
            },
            {
              "subType": "logical_table",
              "name": "ITEM_MNTH_SLS"
            },
            {
              "subType": "logical_table",
              "name": "STATE_REGION_MNTH_SLS"
            },
            {
              "subType": "logical_table",
              "name": "STATE_SUBCATEG_MNTH_SLS"
            },
            {
              "subType": "logical_table",
              "name": "ITEM_CCTR_MNTH_SLS"
            },
            {
              "subType": "logical_table",
              "name": "STATE_SUBCATEG_REGION_SLS"
            },
            {
              "subType": "logical_table",
              "name": "SUBCATEG_MNTH_CTR_SLS"
            },
            {
              "subType": "logical_table",
              "name": "YR_CATEGORY_SLS"
            }
          ]
        },
        {
          "expression": {
            "tokens": [
              {
                "value": "QTY_SOLD * (UNIT_PRICE - DISCOUNT)"
              }
            ]
          },
          "tables": [
            {
              "subType": "logical_table",
              "name": "ORDER_DETAIL"
            }
          ]
        }
      ]
    }

## Create table references

In [ ]:
def get_by_name(list, name):
    for item in list:
        if item.name == name:
            return item
    return None

In [ ]:
def lookup_table_ref(table):
    return SchemaObjectReference(
        name=table.name,
        sub_type=ObjectSubType.LOGICAL_TABLE,
        object_id=table.id,
    )


In [ ]:
def create_lockup_table_refs(conn, tables):
    logical_tables = list_logical_tables(conn)
    table_refs = []
    for t in tables:
        logical_table = get_by_name(logical_tables, t["name"])
        if logical_table == None:
            raise Exception("This should not happen. Logical name referred doesn't.  ")
        else:
            logical_table_ref = lookup_table_ref(logical_table)
        table_refs.append(logical_table_ref)
    return table_refs

## Create Fact Expression

In [ ]:
def create_fact_expression(conn, expression):
    # Use tokens to create fact expression
    tokens_data = expression["expression"]["tokens"]
    tables = create_lockup_table_refs(conn, expression["tables"])
    tokens = []
    for t in tokens_data:
        # type = getTokenType(t["type"])
        # tokens.append(Token(value=t["value"], type=type))
        tokens.append(Token(value=t["value"]))
    fact_exp = FactExpression(
        expression=Expression(tokens=tokens),
        tables=tables
    )
    return fact_exp

## Create Data Type

In [ ]:
def get_data_type(dict):
    type_str = dict["type"]
    type = None
    for t in DataType.Type:
        if t.name == type_str.upper():
            type = t
            break
    if type == None:
        raise Exception("Incorrect data type:" + type_str)
    return DataType(type, dict["precision"], dict["scale"])


## Create Fact

In [ ]:

def create_fact(conn, fact):
    info = fact["information"]
    name = info["name"]
    folder_id = info.get("destinationFolderId")

    objects = list_facts(conn, name=name)
    if len(objects) > 0:
        print("Fact with name:[" + name + "] already exists.")
        return objects[0]
    else:
        expressions = []
        for exp in fact["expressions"]:
            expressions.append(create_fact_expression(conn, exp))
        # expressions = fact["expressions"]
        # for expression in expressions:
        #     expression["tables"] = createLockupTableRefs(conn, expression["tables"])

        fact_obj = Fact.create(conn,
                               name=name,
                               destination_folder=folder_id,
                               description=info.get("description", ""),
                               expressions=expressions,
                               data_type=get_data_type(fact["dataType"]))
        return fact_obj


In [ ]:
fact_obj = create_fact(conn, fact_data)